### Step 1: Load packages

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import scipy.stats
import matplotlib.pyplot as plt
%matplotlib inline
from google.cloud import bigquery
from google.cloud import bigquery_storage
import datetime as dt
import re
import warnings
warnings.filterwarnings(action = 'ignore') # Suppresses pandas warnings
from IPython.display import display
from tqdm.notebook import tqdm_notebook # Displays progress bars when querying data from BQ
from run_sql_queries import run_query_func

### Step 2: Define some input parameters to query the relevant test data

In [2]:
# General inputs
query_path = 'G:\My Drive\APAC\Autopricing\Switchback Testing\switchback_test_automation\sql_queries\data_extraction_queries_automated_script.sql' # This is the path to the BQ script that pulls test data
df_raw_data_tbl_name = 'ab_test_individual_orders_cleaned_switchback_tests' # This is the table that contains the cleaned data of switchback tests
alpha_lvl = 0.05 # Set the alpha level below which p-values are statistically significant
p_val_tbl_id = 'dh-logistics-product-ops.pricing.p_vals_switchback_tests' # The table containing the p-values of metrics. This table will be uploaded to BQ at the end of the script

### Step 3.1: Retrieve the switchback test configurations

In [3]:
client = bigquery.Client(project = 'logistics-data-staging-flat') # Instantiate a BQ client and define the project
bq_storage_client = bigquery_storage.BigQueryReadClient() # Instantiate a BQ storage client

# The switchback_test_configs_bq table gets updated every hour via a scheduled query
sb_test_configs = client.query("""SELECT * FROM `dh-logistics-product-ops.pricing.switchback_test_configs_bq`""")\
    .result()\
    .to_dataframe(bqstorage_client = bq_storage_client, progress_bar_type = 'tqdm_notebook')

Downloading:   0%|          | 0/2 [00:00<?, ?rows/s]

### Step 3.2: Extract the scheme IDs from between the curly brackets

In [4]:
# Apply the extraction function on the "scheme_id_on" and "scheme_id_off" columns
sb_test_configs['scheme_id_on'] = sb_test_configs['scheme_id_on'].apply(lambda x: re.findall('\{(.*?)\}', x)[0])
sb_test_configs['scheme_id_off'] = sb_test_configs['scheme_id_off'].apply(lambda x: re.findall('\{(.*?)\}', x)[0])


### Step 3.3: Create a list of dicts storing the config info of the switchback tests

In [5]:
# Declare an empty dict that will be contain the details of a particular switchback test in each for loop iteration
# The keys of the dict are the column names of sb_test_configs
test_config_dict = {}
keys = list(sb_test_configs.columns)

# Declare an empty list that will contain all the dicts storing the test config information
test_config_lod = []

# Populate the list of dicts (lod) with the test configuration info
for i in range(0, len(sb_test_configs.index)): # Enumerate over the number of tests
    for key in keys: # Populate an intermediary dict with the config info of the test belonging to the current iteration
        test_config_dict[key] = sb_test_configs[key][i]
    test_config_lod.append(test_config_dict) # Append the intermediary dict to the list of dicts
    test_config_dict = {} # Empty the dict so that it can be populated again

### Step 3.4: Amend the structure of the list of dicts so that "zone" and "scheme" columns contain lists instead of strings

In [6]:
for test in range(0, len(test_config_lod)): # Iterate over the test dicts
    for key in ['zone_name_vendor_excl', 'zone_name_customer_excl', 'scheme_id_on', 'scheme_id_off']: # Iterate over these keys specifically to change their contents to a list
        if test_config_lod[test][key] == None: # If the value of the key is None, change it to an empty list
            test_config_lod[test][key] = []
        else: 
            test_config_lod[test][key] = test_config_lod[test][key].split(', ') # Split the components of the string into list elements
        
        if 'scheme_id' in key: # If the key being accessed contains the word "scheme_id", change the components of the list to integers using list comprehension
            test_config_lod[test][key] = [int(sch) for sch in test_config_lod[test][key]]
        else:
            pass

## <center>END OF INPUT SECTION. NO NEED TO ADJUST ANY CODE FROM HERE ONWARDS</center> ##

### Step 4: Run the queries that pull the data of Switchback tests

In [5]:
run_query_func(query_path)

g:\My Drive\APAC\Autopricing\Switchback Testing\Switchback_AA_Tests\venv_switchback_aa_tests\lib\site-packages\google\auth\_default.py:81: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


The SQL script was executed successfully at 2022-08-25 14:58:15.277884 



### Step 5.1: Get the curated data from the resulting table

In [7]:
df_raw_data = client.query("""SELECT * FROM `dh-logistics-product-ops.pricing.{}`"""\
    .format(df_raw_data_tbl_name))\
    .result()\
    .to_dataframe(bqstorage_client = bq_storage_client, progress_bar_type = 'tqdm_notebook')

Downloading:   0%|          | 0/296226 [00:00<?, ?rows/s]

### Step 5.2: Change the data types of columns in the dataset

In [8]:
# Define the start of the data frame where the data types of columns need to be changed 
col_start = np.where(df_raw_data.columns == 'exchange_rate')[0][0]

# Change data types --> df[df.cols = specific cols].apply(pd.to_numeric)
df_raw_data[df_raw_data.columns[col_start:]] = df_raw_data[df_raw_data.columns[col_start:]].apply(pd.to_numeric, errors = 'ignore')

### Step 6: Filter the data frame for the relevant data, add a few supplementary columns, then calculate the agg metrics and p-values

In [9]:
# Define the list of KPIs
col_list = [
    'actual_df_paid_by_customer', 'gfv_local', 'gmv_local', 'commission_local', 'joker_vendor_fee_local', # Customer KPIs (1)
    'sof_local', 'service_fee_local', 'revenue_local', 'delivery_costs_local', 'gross_profit_local', # Customer KPIs (2)
    'dps_mean_delay', 'delivery_distance_m', 'actual_DT' # Logistics KPIs
]

# Initialize empty lists for the total/per order metrics and p-values. These will be later changed to data frames
df_final_per_order = []
df_final_tot = []
df_final_pval = []

# Iterate over the original data frame, filtering for the relevant test data every time and computing the per-order metrics
for i in range(0, len(test_config_lod)):
    df_temp = df_raw_data[
        (df_raw_data['target_group'] != 'Non_TG') &
        (df_raw_data['test_name'] == test_config_lod[i]['test_name']) &
        (~ df_raw_data['zone_name_vendor'].isin(test_config_lod[i]['zone_name_vendor_excl'])) &
        (~ df_raw_data['zone_name_customer'].isin(test_config_lod[i]['zone_name_customer_excl'])) &
        ((df_raw_data['scheme_id'].isin(test_config_lod[i]['scheme_id_on'])) | (df_raw_data['scheme_id'].isin(test_config_lod[i]['scheme_id_off']))) &
        (df_raw_data['order_placed_at_local'].dt.date.between(test_config_lod[i]['test_start'], test_config_lod[i]['test_end']))
    ] # Filter out Non_TG orders because they will contain irrelevant price schemes

    # We will add a supplementary column to "df_temp" in the for loop below, so we need to create a function with the conditions
    on_off_conditions = [
        (df_temp['scheme_id'].isin(test_config_lod[i]['scheme_id_on'])),
        (df_temp['scheme_id'].isin(test_config_lod[i]['scheme_id_off']))
    ]

    # Add a supplementary column to indicate if the order belonged to an 'On' or 'Off' day
    df_temp['on_or_off_day'] = np.select(on_off_conditions, ['On', 'Off'])

    # Calculate the "per order" metrics and rename the column label to "df_per_order_metrics"
    df_per_order = round(df_temp.groupby(['test_name', 'on_or_off_day'])[col_list].mean(), 2)
    df_per_order = df_per_order.rename_axis(['df_per_order_metrics'], axis = 1)
    
    # Calculate the "total" metrics and rename the column label to "df_tot_metrics"
    df_tot = round(df_temp.groupby(['test_name', 'on_or_off_day'])[col_list[:-3]].sum(), 2) # [:-3] excludes the logistics KPIs
    df_tot = df_tot.rename_axis(['df_tot_metrics'], axis = 1)

    # Append "df_per_order" and "df_tot" for the for loop's iteration to the previously initialized variables "df_final_per_order" and "df_tot"
    df_final_per_order.append(df_per_order)
    df_final_tot.append(df_tot)

    # Now, we need to calculate the p-values. Create two sub-data frames for the 'On' and 'Off' days
    df_on_days = df_temp[df_temp['on_or_off_day'] == 'On']
    df_off_days = df_temp[df_temp['on_or_off_day'] == 'Off']

    pval_dict = {} # Initialize an empty dict that will contain the p-value of each KPI
    for i in col_list:
        pval = round(scipy.stats.mannwhitneyu(x = df_on_days[i], y = df_off_days[i], alternative = 'two-sided', nan_policy = 'omit')[1], 4)
        pval_dict[i] = pval
    
    df_final_pval.append(pval_dict) # df_final_pval is a list of dicts

# Concatenate "df_final_per_order" from all tests into one data frame and display it
df_final_per_order = pd.concat(df_final_per_order)
display(df_final_per_order)

# Concatenate "df_final_tot" from all tests into one data frame
df_final_tot = pd.concat(df_final_tot)
# Add a thousand separator
for i in df_final_tot.columns:
    df_final_tot[i] = df_final_tot[i].map('{:,}'.format)
# Display the data frame
display(df_final_tot)

# Change the list of dicts "df_final_pval" to a data frame
df_final_pval = pd.DataFrame(df_final_pval)\
    .assign(
        test_name = [iter['test_name'] for iter in test_config_lod],
        upload_timestamp = dt.datetime.now()
    )\
    .set_index('test_name')

# Define a function that changes the font color based on the p-value
def color_sig_green(val):
    """
    Takes a scalar and returns a string with the css property 'color: green' for statistically significant values, red otherwise
    """
    if val <= alpha_lvl:
        color = 'green'
    else:
        color = 'red'
    return 'color: %s' % color

df_final_pval.iloc[:, :-1].style.format(precision = 4).applymap(color_sig_green)

df_per_order_metrics                                              actual_df_paid_by_customer  \
test_name                                          on_or_off_day                               
TH_20220721_R_F0_O_Switchback_AA_Test_Trang        Off                                  6.37   
                                                   On                                   6.32   
PH_20220721_R_F0_O_SB_AA_Test_Calamba_No_Condition Off                                 40.19   
                                                   On                                  40.13   

df_per_order_metrics                                              gfv_local  \
test_name                                          on_or_off_day              
TH_20220721_R_F0_O_Switchback_AA_Test_Trang        Off               109.83   
                                                   On                110.79   
PH_20220721_R_F0_O_SB_AA_Test_Calamba_No_Condition Off               371.85   
                                                   On                373.36   

df_per_order_metrics                                              gmv_local  \
test_name                                          on_or_off_day              
TH_20220721_R_F0_O_Switchback_AA_Test_Trang        Off               111.14   
                                                   On                111.77   
PH_20220721_R_F0_O_SB_AA_Test_Calamba_No_Condition Off               381.48   
                                                   On                383.39   

df_per_order_metrics                                              commission_local  \
test_name                                          on_or_off_day                     
TH_20220721_R_F0_O_Switchback_AA_Test_Trang        Off                       31.95   
                                                   On                        32.21   
PH_20220721_R_F0_O_SB_AA_Test_Calamba_No_Condition Off                       77.38   
                                                   On                        77.97   

df_per_order_metrics                                              joker_vendor_fee_local  \
test_name                                          on_or_off_day                           
TH_20220721_R_F0_O_Switchback_AA_Test_Trang        Off                              0.39   
                                                   On                               0.39   
PH_20220721_R_F0_O_SB_AA_Test_Calamba_No_Condition Off                              0.85   
                                                   On                               1.21   

df_per_order_metrics                                              sof_local  \
test_name                                          on_or_off_day              
TH_20220721_R_F0_O_Switchback_AA_Test_Trang        Off                 0.15   
                                                   On                  0.15   
PH_20220721_R_F0_O_SB_AA_Test_Calamba_No_Condition Off                 0.49   
                                                   On                  0.51   

df_per_order_metrics                                              service_fee_local  \
test_name                                          on_or_off_day                      
TH_20220721_R_F0_O_Switchback_AA_Test_Trang        Off                          0.0   
                                                   On                           0.0   
PH_20220721_R_F0_O_SB_AA_Test_Calamba_No_Condition Off                          0.0   
                                                   On                           0.0   

df_per_order_metrics                                              revenue_local  \
test_name                                          on_or_off_day                  
TH_20220721_R_F0_O_Switchback_AA_Test_Trang        Off                    38.86   
                                                   On                     39.07   
PH_20220721_R_F0_O_SB_AA_Test_Calamba_No_Condition Off                   

df_tot_metrics                                                   actual_df_paid_by_customer  \
test_name                                          on_or_off_day                              
TH_20220721_R_F0_O_Switchback_AA_Test_Trang        Off                            452,854.0   
                                                   On                            445,338.12   
PH_20220721_R_F0_O_SB_AA_Test_Calamba_No_Condition Off                           381,462.46   
                                                   On                            376,469.68   

df_tot_metrics                                                       gfv_local  \
test_name                                          on_or_off_day                 
TH_20220721_R_F0_O_Switchback_AA_Test_Trang        Off             7,807,004.0   
                                                   On              7,804,615.0   
PH_20220721_R_F0_O_SB_AA_Test_Calamba_No_Condition Off            3,529,624.25   
                                                   On             3,502,882.35   

df_tot_metrics                                                       gmv_local  \
test_name                                          on_or_off_day                 
TH_20220721_R_F0_O_Switchback_AA_Test_Trang        Off            7,900,551.12   
                                                   On             7,873,499.08   
PH_20220721_R_F0_O_SB_AA_Test_Calamba_No_Condition Off            3,621,029.26   
                                                   On             3,596,930.28   

df_tot_metrics                                                   commission_local  \
test_name                                          on_or_off_day                    
TH_20220721_R_F0_O_Switchback_AA_Test_Trang        Off               2,271,192.04   
                                                   On                2,268,949.19   
PH_20220721_R_F0_O_SB_AA_Test_Calamba_No_Condition Off                  734,498.5   
                                                   On                  731,528.26   

df_tot_metrics                                                   joker_vendor_fee_local  \
test_name                                          on_or_off_day                          
TH_20220721_R_F0_O_Switchback_AA_Test_Trang        Off                         27,700.5   
                                                   On                          27,233.0   
PH_20220721_R_F0_O_SB_AA_Test_Calamba_No_Condition Off                          8,077.0   
                                                   On                          11,374.5   

df_tot_metrics                                                   sof_local  \
test_name                                          on_or_off_day             
TH_20220721_R_F0_O_Switchback_AA_Test_Trang        Off            10,817.0   
                                                   On             10,818.0   
PH_20220721_R_F0_O_SB_AA_Test_Calamba_No_Condition Off             4,686.0   
                                                   On              4,806.0   

df_tot_metrics                                                   service_fee_local  \
test_name                                          on_or_off_day                     
TH_20220721_R_F0_O_Switchback_AA_Test_Trang        Off                         0.0   
                                                   On                          0.0   
PH_20220721_R_F0_O_SB_AA_Test_Calamba_No_Condition Off                         0.0   
                                                   On                          0.0   

df_tot_metrics                                                   revenue_local  \
test_name                                          on_or_off_day                 
TH_20220721_R_F0_O_Switchback_AA_Test_Trang        Off            2,762,563.54   
                                                   On             2,752,338.31   
PH_20220721_R_F0_O_SB_AA_Test_Calamba_No_Condition Off            1,128

,actual_df_paid_by_customer,gfv_local,gmv_local,commission_local,joker_vendor_fee_local,sof_local,service_fee_local,revenue_local,delivery_costs_local,gross_profit_local,dps_mean_delay,delivery_distance_m,actual_DT
test_name,,,,,,,,,,,,,
TH_20220721_R_F0_O_Switchback_AA_Test_Trang,0.0098,0.0072,0.0620,0.0067,0.6312,0.9503,1.0000,0.0266,0.3132,0.0035,0.0000,0.0295,0.0009
PH_20220721_R_F0_O_SB_AA_Test_Calamba_No_Condition,0.5124,0.2979,0.1205,0.8089,0.3342,0.2915,1.0000,0.7717,0.1811,0.2939,0.5822,0.9245,0.2712


### Step 7: Upload the data frame containing the p-values to BQ

In [10]:
# Since string columns use the "object" dtype, pass in a (partial) schema to ensure the correct BigQuery data type.
job_config = bigquery.LoadJobConfig(schema = [
    bigquery.SchemaField('test_name', 'STRING'),
])

# Set the job_config to overwrite the data in the table
job_config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE

# Upload the p-values dataframe to BQ 
job = client.load_table_from_dataframe(
    dataframe = df_final_pval.reset_index(),
    destination = p_val_tbl_id,
    job_config = job_config
)

job.result() # Wait for the load job to complete

LoadJob<project=logistics-data-staging-flat, location=US, id=1671a399-1841-4746-ae25-de4ee8f3907c>